In [96]:
R3.<x, y, z> = GF(3)[]

def kronecker_map(f, n, R):
    vars = R.gens()
    
    kmap = {}
    p = 1
    
    for v in vars:
        kmap[v] = R.gen(0)^p
        p *= n
        
    return f.subs(kmap)

def inv_kronecker_map(f, n, R):
    inv_f = 0
    vars = R.gens()
    
    for coeff, monom in list(f):
        inv_term = coeff
        
        cur_var_idx = 0
        deg = int(monom.degree(vars[0]))
        
        while deg > 0:
            inv_term *= vars[cur_var_idx] ^(deg % n)
            deg //= n
            cur_var_idx += 1
        
        inv_f += inv_term
            
    return inv_f
    
    
def factorize(f):
    H = list(f.factor())
    # Constructing the list of elements to choose from
    elements = []
    for factor, multiplicity in H:
        elements.extend([factor] * multiplicity)
        
    return elements

In [103]:
from sage.combinat.combination import Combinations

def kronecker_get_factor(f, R):
    degs = [f.degree(var) for var in R.gens()]
    n = max(degs)+1
    print("Max Degree: ", n)
    kf = kronecker_map(f, n, R)
    print("Applying Kronecker Map: ", kf)
    H = factorize(kf)
    
    print("Factors after kronecker map: ", H)
    G = []
    
    for i in range(1, len(H)//2+1):
        for S in Combinations(H, i):
            g = 1
            
            for t in S:
                g *= t
            
            inv_g = inv_kronecker_map(g, n, R)
            
            if inv_g.divides(f):
                print("Recovered factor: ", inv_g)
                return inv_g
    print("Given f is irreducible,, returning factor: ", f)
    return f
    



In [107]:
f = - x^4 * y + x^3 * z + x * z^2 + y * z^2
f in R3

G = []

while f != 1:
    print("Factorizing polynomial: ", f)
    g = kronecker_get_factor(f, R3)
    f = f //g
    G.append(g)
    print("\n")
    
print("All factors: ", G)

Factorizing polynomial:  -x^4*y + x^3*z + x*z^2 + y*z^2
Max Degree:  5
Applying Kronecker Map:  x^55 + x^51 + x^28 - x^9
Factors after kronecker map:  [x + 1, x + 1, x + 1, x, x, x, x, x, x, x, x, x, x^5 + x^4 + x^3 - x^2 + x - 1, x^11 + x^8 + x^6 - x^4 + x^3 + x^2 - x + 1, x^11 - x^10 + x^9 + x^8 - x^7 + x^5 + x^3 + 1, x^16 - x^14 + x^13 - x^12 + x^11 - x^9 + x^7 - x^6 - x^5 - x^3 + x^2 - x + 1]
Recovered factor:  x^2 + z


Factorizing polynomial:  -x^2*y + x*z + y*z
Max Degree:  3
Applying Kronecker Map:  x^12 + x^10 - x^5
Factors after kronecker map:  [x + 1, x + 1, x, x, x, x, x, x^2 + x - 1, x^3 - x + 1]
Given f is irreducible,, returning factor:  -x^2*y + x*z + y*z


All factors:  [x^2 + z, -x^2*y + x*z + y*z]
